In [1]:
import pandas as pd
import numpy as np
from glob import glob


##     LOAD FILES     ##
# Input Folder (the folder where the annotations are):
file_paths = glob(r"C:\Users\Michael's  Laptop\Documents\XSELL\ATT\QABOT\unsent to DS (9.17)/*.csv")
df = pd.concat([pd.read_csv(file_path,error_bad_lines=False) for file_path in file_paths])
print "File paths loaded:"
print file_paths
# Output directory (Where you want the split files to be):
directory = r"C:\Users\Michael's  Laptop\Documents\XSELL\Done (sent to DS 6.5)/"


##     CHECK FILES     ##
#total number of chats in the input folder 
#(If number is off from what you epexect )
print("")
print("Checking total chats and annotations...")
print('Total Chats: ' + str(df.engagement_id.nunique()))

#Total number of annotations
#(Should match total number of chats, is okay if off by 1 or 2)
print("Total Annotations: "+ str(df.Topic.count()))

print("")
continue_q = raw_input("Do you want to check for duplicates and split files? (y/n): ")
if continue_q == 'n':
    exit()




File paths loaded:
["C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_7.30 - AA_EL-DONE.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_7.31 - KB_NN - DONE.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.10-KO-DONE.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.12 el - DONE.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.13-KB-done.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.15-KO - DONE.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.16-BA-DONE.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.19 el done.csv", "C:\\Users\\Michael's  Laptop\\Documents\\XSELL\\ATT\\QABOT\\unsent to DS (9.17)\\buca_8.9-FB-DONE.csv"]

Checking t

c:\python27\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


AttributeError: 'DataFrame' object has no attribute 'Topic'

In [ ]:
##     CLEAN DATA FOR ANNOTATION AND SPLIT     ##

#This function is designed to remove any duplicates in the given dataframe
#(Thomas from data science wrote this if you have any questions)
print("")
print("Removing Duplicates...")
def remove_dupes(df, exclude_ids=None):
    
    unchecked = set(df.engagement_id.unique()) 
    if exclude_ids:
        unchecked = unchecked - set(exclude_ids)
    print("Number of unique chat ID's: " + str(len(unchecked)))
    current_id = ''
    rows = []
    for row in df.itertuples():
        if row.engagement_id != current_id:
            # we should be done with current_id
            if current_id in unchecked:
                unchecked.remove(current_id)
            # check if this engagement id hasn't been seen yet
            if row.engagement_id in unchecked:
                current_id = row.engagement_id
            else:
                current_id = ''
        if row.engagement_id == current_id:
            rows.append(row)

    print('Number of duplicate chats (should be 0):'+ str(len(unchecked)))
    return pd.DataFrame(rows)
# create new dataframe running remove duplication function over old dataframe
df2 = remove_dupes(df)
df = df2

#The agent_id and chat_id should not be a problem for verizon but might be for at&T. Helps solve a problem where 
# agent id's aren't on every line of a chat
agent_id = 'agent_id'
chat_id = 'engagement_id'
# fill empty values in agent id column based on non-empty values in the same chat
df[agent_id] = df.groupby([chat_id], sort=False)[agent_id].apply(lambda x: x.ffill().bfill())


## SPLIT THE CHATS INTO EQUAL CHUNKS     ##
#The below variable is how many chats you want per file. Change to whatever you want, 500 is standard and should run on most computers
print("")
print("Splitting chats into equal chunks...")
chats_per_file = 500
chat_ids = list(df[chat_id].unique())
chat_chunks = [chat_ids[i:i + chats_per_file] for i in range(0, len(chat_ids), chats_per_file)]
print('Total Chat IDs: {}'.format(len(chat_ids)))
print('Chunks: {}'.format(len(chat_chunks)))
print('Length of individual chunks:')
for  i in chat_chunks:
    print  len(i)
    
##     DOWNLOAD FILES AS CSV'S TO OUTPUT FOLDER     ##
print('')
print("Saving files in the following locations...")
for i, chat_chunk in enumerate(chat_chunks):
    filename = (directory + 'verizon_dataset_10_chunk_{}.csv'.format(i))
    print(filename)
    df[df[chat_id].isin(chat_chunk)].to_csv(filename, index=False)